# Welcome

### What was previously covered
If you are following my work, PERSON WHO FORKED MY NOTEBOOK, I SEE YOU AND I APPRECIATE YOU, you will know that in the previous notebook I covered the general statistics of the variables, definied what the variables meant to me, and explored a few other assumptions.
The assumptions are as following: 
- Were women and children ushered off first?
- Fare, and Class demonstrate wealth, which wealth really mattered?
- Did people in the same PassengerClass pay the same fare?
- What does the cabin variable mean?

What I found was from the data was that:
Women were ushered off first, but I cannot be certain if children were ushered off first. Both fare paid and passenger class were related to survivability where the richer were more likely to survive, and that the passenger class correlation mattered more than fare. People in the same class did not pay the same fare, but across all classes, many paid the minimum amount. And finally, the cabin variable represents locations that the passenger resided in, which can be mapped to a map of the Titanic if someone pleased to do so. 

A link to the previous notebook can be accessed below:    
https://www.kaggle.com/karagain/karsten-s-titanic-exploration-notebook

### What will be covered in this Titanic Notebook
In this notebook I will explore the following areas in this dataset:
- The survivability of different classes that paid the same fare. 
- Variable exploration, if I were to use my current knowledge and statistical knowledge, what would I use to build a model?
- A basic model that uses Heuristics to predict survivability as baselines.
- A model that beats the heuristic models. 

In [2]:
# Data Processing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Classifiers
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics

# Package to Access Dataset
import os

In [3]:
# Import training Dataset
df = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
full = pd.concat([df, test])

# Why these Classifiers?
If you looked at the packages imported, you would see that I have chosen two classifiers, GaussianNB and LogisticRegression. Both are classifiers that can spit out a binary result whether the individual survives or not. The simplest difference between the two is the Naive Bayes is based on the joint probability (so the prior probability and posterior probability) of the factors in the training data, Logistic Regression is a "linear" regression where the classification is determined by the decision boundary of y = 0.5. Since Logistic Regression is linear, the model is fitted by minimizing the squared error of the regression to the training data, and the variables have an "All Else Equal" effect on the probability. 

These two classifiers are easy to understand, easy to tune, and are great for learning machine learning. I plan on using these classifiers for a baseline comparison if I participate in the competition. 

# Cont. EDA from the last notebook!
## The survivability of different classes that paid the same fare
The last time we were exploring was in the previous notebook where we were left on a cliffhanger. This time I will explore a 3+ dimensional result to really isolate the effect of class on survivability, with fare kept equal

In [4]:
# Look at the relative distribution for the Fare
df.Fare.hist(bins=70)

In [5]:
# Get a statistical view of how the results are distributed. 
df.Fare.describe()

In [6]:
plt.figure(figsize=(6,6))
fare = 5
df.Pclass[df.Fare<=fare].hist()
plt.title('Price Less Than ' + str(fare))
plt.ylabel('Tickets Purchased')

From this most recent graph, we can see that there are some people who paid no fare for the results. On top of that, from this article published by Time about the most expensive ticket, we can see that there are other variables that influenced survivability. Servants of very affluent individuals were also able to escape despite their lower class. I feel that any model that I would  conduct on this table will be extremely inaccurate. I will keep that in mind and come back to it at the end of this notebook. 

Time Link: http://time.com/money/4283738/titanic-most-expensive-ticket/

In [7]:
sns.set_style(style='whitegrid')
for x in [5, 10, 50, 100, 500, 1000]:
    sns.factorplot("Pclass", "Survived", hue="Sex", data=df[df.Fare<=x], kind="bar", ci=None).fig.suptitle('Fare less than ' + str(x))
# sns.factorplot("Pclass", "Survived", hue="Sex", data=df[df.Fare<=50], kind="bar", ci=None).fig.suptitle('Fare less than 50')


The following sets of graphs show the probability of that gender surviving given that they paid in a certain level. The less than 1000 fare should encompass what we have covered before, showing that more than 90% of women survived if they were upper or middle class, while men had consistently low levels of less than 40% surviving for any of the classes. 

We saw from an earlier graph of how many people made purchases of tickets, but I was unable to combine those graphs together. I decided not to do so since I have already spent upwards of an hour trying to figure it out, and I feel that my time could be better allocated. 
The earlier graph shows that there were people making purchases on low fared tickets, but they did not survive despite being of higher class. 

If you tried to check out the results, in the exploratory graph below, the magical numbers for these critical points seem to be at 11 for the middle class, and 27 for the upper class. 

Overall the information is conflicting with what is available online, which suggests the price for first class tickets were minimum 30 pounds. 

In conclusion, if we compare the results form the fare ticket being 30 pounds, versus 100 pounds, the survivability probability actually stays mostly the same. The women's probability increased from a high 80% to a low 90% but this can just be noise in the results. So from this analysis on the extended brackets of fare paid and survivability separated by classes, no fare was not a factor in surviving. It makes sense as the price of a ticket is not a great indicator of a person's wealth, compared to what class they belong to, or their associated companies. 

In [8]:
# Changing the x variable allows you to view the tickets purchased below that price, and the class survivability probability. 
x = 50

df.Pclass[df.Fare<=x].hist()
plt.title('Price Less Than ' + str(x))
plt.ylabel('Tickets Purchased')

sns.factorplot("Pclass", "Survived", hue="Sex", data=df[df.Fare<=x], kind="bar", ci=None).fig.suptitle('Fare less than ' + str(x))


# Model Building
Given what we have covered so far, there are a few strong indicators for survivability to build a model. We could use the class, the sex, and the cabins. We will not use the cabins because too many rows of information are missing. 

Let's build two simple models.

In [ ]:
# Change the strings into numerals so the data can be recognized by the classifiers
df.Sex = df.Sex.str.replace('female', '0')
df.Sex = df.Sex.str.replace('male', '1')
df.Sex = df.Sex.astype('int')

In [19]:
# Separate the two testing cases
train = df[['Pclass', 'Sex']]

# Train and Run the GaussianNB classifier
features_train, features_test, labels_train, labels_test = train_test_split(train, df.Survived, test_size = 0.3, random_state = 42)
clf = GaussianNB()
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

accuracy_score(labels_test, pred)

In [20]:
clf = LogisticRegression()
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

accuracy_score(labels_test, pred)

## Qualification
For the two classifiers, we yielded the same accuracy. The 80% accuracy for the test set seems pretty good, much higher than a 50% for a truly random guess for a binary result. But is it really?

Recall from the previous notebook, we saw that majority of the survivors were female, or upper class. We expected the classifier to do better than random guesses, but how do the results fare against heuristics?

In [21]:
# We recall that from the training data, 577 are men, 314 are female. 
df.Sex.value_counts()

In [26]:
# Visualizations; These graphs show that a large portion of women, and a little portion of men survived. 
plt.figure(figsize=(13,10))
plt.subplot(231)
plt.title('Female Survivors')
plt.bar(['Survived', 'Deceased'],df.Survived[df.Sex == 0].value_counts(normalize=True), color = ['c', 'm'])

plt.subplot(232)
plt.title('Male Survivors')
plt.bar(['Deceased', 'Survived'],df.Survived[df.Sex == 1].value_counts(normalize=True), color = ['r', 'b'])

In [32]:
# Here I create a new column to compare to Survived
df.loc[df.Sex == 0, 'pred'] = 1
df.loc[df.Sex == 1, 'pred'] = 0

# Get an accuracy score for those survived and this heuristic test
accuracy_score(df.Survived, df.pred)

The results show that our guess was negligly better than a heuristic. The two classifiers did better by half a percent. 

# Takeaways
So in this notebook, we did some more exploring for a question that was thought up in the previous notebook. We created a model, and we compared it to a simple guess model used as a baseline. From this we learned that we need to really improve our analytical ability to make sure we get much better results than two obvious results.

# Next Notebook
To improve upon this result, the next notebook will:
- Explore variables more deeply. Run a generic correlation graph
- Clean the cabin data so the results are usable
- Fix the age variable by manually inputting data from online
- Run a few different models with two more variables and compare. 
- Create another heuristic model that will include a second pivot of class. 